In [ ]:
import sys
sys.path.append('/host/d/Github')
import os
from dipy.align.reslice import reslice
import numpy as np
import nibabel as nb
import Registraion_4DCT_Leyu.Data_process as dp
import Registraion_4DCT_Leyu.functions_collection as ff

## step 1: rename original file

In [15]:
dataset = 'Popi'
data_path = os.path.join('/host/d/Data/4DCT',dataset)
print('data path: ', data_path)

case_list = ff.sort_timeframe(ff.find_all_target_files(['C*'],data_path),0,'e')
print('case_list:', case_list, 'how many cases:', len(case_list))

data path:  /host/d/Data/4DCT/Popi
case_list: ['/host/d/Data/4DCT/Popi/Case1' '/host/d/Data/4DCT/Popi/Case2'
 '/host/d/Data/4DCT/Popi/Case3' '/host/d/Data/4DCT/Popi/Case4'
 '/host/d/Data/4DCT/Popi/Case5' '/host/d/Data/4DCT/Popi/Case6'] how many cases: 6


In [16]:
for case_num in range(len(case_list)):
    case_path = case_list[case_num]
    print('processing case: ', case_path)

    case_num = ff.find_timeframe(case_path,0,'e')
    print('this is case number: ', case_num)

    original_image_folder = os.path.join(case_path, 'original_image')

    # total_timeframes = len(ff.find_all_target_files(['ca*'],original_image_folder)) # this is for DIR-LAB
    total_timeframes = len(ff.find_all_target_files(['img_*'],original_image_folder)) # this is for Popi
    print('total timeframes: ', total_timeframes)

    for t in range(total_timeframes):
        # print('processing timeframe: ', t)
        if os.path.isfile(os.path.join(original_image_folder, 'img_'+str(t)+'.nii.gz')):
            print('file img_'+str(t)+'.nii.gz already exists, skip renaming')
            continue

        # original_file_name = 'case'+str(case_num)+'_T'+str(t)+'0_s.nii.gz' # this is for DIR-LAB
        original_file_name = 'img_'+str(t)+'0.nii.gz'  # this is for Popi
        new_file_name = 'img_'+str(t)+'.nii.gz'
        # print('renaming file: ', original_file_name, ' to ', new_file_name)

        # rename
        os.rename(os.path.join(original_image_folder, original_file_name),
                  os.path.join(original_image_folder, new_file_name))


processing case:  /host/d/Data/4DCT/Popi/Case1
this is case number:  1
total timeframes:  10
processing case:  /host/d/Data/4DCT/Popi/Case2
this is case number:  2
total timeframes:  10
processing case:  /host/d/Data/4DCT/Popi/Case3
this is case number:  3
total timeframes:  10
processing case:  /host/d/Data/4DCT/Popi/Case4
this is case number:  4
total timeframes:  10
processing case:  /host/d/Data/4DCT/Popi/Case5
this is case number:  5
total timeframes:  10
processing case:  /host/d/Data/4DCT/Popi/Case6
this is case number:  6
total timeframes:  10


## Process image

In [17]:
dataset = 'DIR_lAB'
data_path = os.path.join('/host/d/Data/4DCT',dataset)
print('data path: ', data_path)

case_list = ff.sort_timeframe(ff.find_all_target_files(['C*'],data_path),0,'e')
print('case_list:', case_list, 'how many cases:', len(case_list))

data path:  /host/d/Data/4DCT/DIR_lAB
case_list: ['/host/d/Data/4DCT/DIR_lAB/Case1' '/host/d/Data/4DCT/DIR_lAB/Case2'
 '/host/d/Data/4DCT/DIR_lAB/Case3' '/host/d/Data/4DCT/DIR_lAB/Case4'
 '/host/d/Data/4DCT/DIR_lAB/Case5' '/host/d/Data/4DCT/DIR_lAB/Case6'
 '/host/d/Data/4DCT/DIR_lAB/Case7' '/host/d/Data/4DCT/DIR_lAB/Case8'
 '/host/d/Data/4DCT/DIR_lAB/Case9' '/host/d/Data/4DCT/DIR_lAB/Case10'] how many cases: 10


In [ ]:
for case_num in range(len(case_list)):
    case_path = case_list[case_num]
    print('processing case: ', case_path)

    # define save folder
    resampled_folder = os.path.join(case_path, 'resampled_image')
    cropped_folder = os.path.join(case_path, 'cropped_image')

    ff.make_folder([resampled_folder, cropped_folder])

    # fine how many time frames
    total_timeframes = len(ff.find_all_target_files(['img_*'],os.path.join(case_path, 'original_image')))
    print('total timeframes: ', total_timeframes)

    for t in range(total_timeframes):
        original_image_file = os.path.join(case_path, 'original_image', 'img_'+str(t)+'.nii.gz')

        # load
        nii_img = nb.load(original_image_file)
        affine = nii_img.affine
        # get current pixel dimension
        pixdim = nii_img.header.get_zooms()
        # print('current pixel dimension:', pixdim)
        # print('current image dimension:', nb.load(original_image_file).shape)

        # resample to [1.25, 1.25, 2.5] mm
        new_dim = [1.25,1.25,2.5]
        nii_img_resampled = dp.resample_nifti(nii_img, order=3, mode='nearest', cval=np.min(nii_img.get_fdata()), in_plane_resolution_mm=new_dim[0], slice_thickness_mm=new_dim[-1])
    
        # turn image from float to int
        data_resampled = nii_img_resampled.get_fdata()
        data_resampled = np.round(data_resampled).astype(np.int16)
        nii_img_resampled = nb.Nifti1Image(data_resampled, nii_img_resampled.affine, nii_img_resampled.header)

        # save resampled image
        resampled_image_file = os.path.join(resampled_folder, 'img_'+str(t)+'.nii.gz')
        nb.save(nii_img_resampled, resampled_image_file)
        print('resampled image saved to: ', resampled_image_file)

        # crop image to [256, 256, 128]
        target_size = [256, 256, 128]
        # load resampled image
        nii_img_resampled = nb.load(resampled_image_file).get_fdata()
        nii_img_cropped = dp.crop_or_pad(nii_img_resampled, target_size, np.min(nii_img_resampled))
        # make sure it's int16
        nii_img_cropped = np.round(nii_img_cropped).astype(np.int16)
        # save
        nii_img_cropped = nb.Nifti1Image(nii_img_cropped, affine, nii_img.header)
        cropped_image_file =  os.path.join(cropped_folder, 'img_'+str(t)+'.nii.gz')
        nb.save(nii_img_cropped, cropped_image_file)
        print('cropped image saved to: ', cropped_image_file)
       
    

processing case:  /host/d/Data/4DCT/DIR_lAB/Case1
total timeframes:  10
resampled image saved to:  /host/d/Data/4DCT/DIR_lAB/Case1/resampled_image/img_0.nii.gz
cropped image saved to:  /host/d/Data/4DCT/DIR_lAB/Case1/cropped_image/img_0.nii.gz
resampled image saved to:  /host/d/Data/4DCT/DIR_lAB/Case1/resampled_image/img_1.nii.gz
cropped image saved to:  /host/d/Data/4DCT/DIR_lAB/Case1/cropped_image/img_1.nii.gz
resampled image saved to:  /host/d/Data/4DCT/DIR_lAB/Case1/resampled_image/img_2.nii.gz
cropped image saved to:  /host/d/Data/4DCT/DIR_lAB/Case1/cropped_image/img_2.nii.gz
resampled image saved to:  /host/d/Data/4DCT/DIR_lAB/Case1/resampled_image/img_3.nii.gz
cropped image saved to:  /host/d/Data/4DCT/DIR_lAB/Case1/cropped_image/img_3.nii.gz
resampled image saved to:  /host/d/Data/4DCT/DIR_lAB/Case1/resampled_image/img_4.nii.gz
cropped image saved to:  /host/d/Data/4DCT/DIR_lAB/Case1/cropped_image/img_4.nii.gz
resampled image saved to:  /host/d/Data/4DCT/DIR_lAB/Case1/resampled